In [92]:
# module imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import random
from sklearn.decomposition import PCA

# model imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import KernelPCA


# processing imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef, precision_score, recall_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score,classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(1635848)

In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
file_path_full_training_set = '/content/drive/MyDrive/KDD-NSL/KDDTrain+.txt'
file_path_test = '/content/drive/MyDrive/KDD-NSL/KDDTest+.txt'

In [95]:
df = pd.read_csv(file_path_full_training_set)
test_df = pd.read_csv(file_path_test)

In [96]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

In [97]:
df.columns = columns
test_df.columns = columns

In [98]:
# Categorical features analysis
print('Training set categorical features:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Training set categorical features:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'attack' has 23 categories


In [99]:
# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Training set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 70 categories
Feature 'flag' has 11 categories
Feature 'attack' has 23 categories


In [100]:
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())

Distribution of categories in service:
service
http        40338
private     21853
domain_u     9043
smtp         7313
ftp_data     6859
Name: count, dtype: int64


In [101]:
# Test set
print('Test set:')
for col_name in test_df.columns:
    if test_df[col_name].dtypes == 'object' :
        unique_cat = len(test_df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

Test set:
Feature 'protocol_type' has 3 categories
Feature 'service' has 64 categories
Feature 'flag' has 11 categories
Feature 'attack' has 38 categories


In [102]:
# Attack Distribution Comparison Analysis
print("="*60)
print("ATTACK TYPE DISTRIBUTION ANALYSIS")
print("="*60)

print("\nTraining Set Attack Distribution:")
print(df['attack'].value_counts().head(10))

print("\nTest Set Attack Distribution:")
print(test_df['attack'].value_counts().head(10))

print("\nUnique Attacks Comparison:")
train_attacks = set(df['attack'].unique())
test_attacks = set(test_df['attack'].unique())

print(f"Training attacks: {len(train_attacks)} unique types")
print(f"Test attacks: {len(test_attacks)} unique types")

attacks_only_in_train = train_attacks - test_attacks
attacks_only_in_test = test_attacks - train_attacks
common_attacks = train_attacks.intersection(test_attacks)

print(f"\nAttacks ONLY in training: {len(attacks_only_in_train)}")
if attacks_only_in_train:
    print(f"   {list(attacks_only_in_train)}")

print(f"\nAttacks ONLY in test: {len(attacks_only_in_test)}")
if attacks_only_in_test:
    print(f"   {list(attacks_only_in_test)}")

print(f"\nCommon attacks: {len(common_attacks)}")

ATTACK TYPE DISTRIBUTION ANALYSIS

Training Set Attack Distribution:
attack
normal         67342
neptune        41214
satan           3633
ipsweep         3599
portsweep       2931
smurf           2646
nmap            1493
back             956
teardrop         892
warezclient      890
Name: count, dtype: int64

Test Set Attack Distribution:
attack
normal          9711
neptune         4656
guess_passwd    1231
mscan            996
warezmaster      944
apache2          737
satan            735
processtable     685
smurf            665
back             359
Name: count, dtype: int64

Unique Attacks Comparison:
Training attacks: 23 unique types
Test attacks: 38 unique types

Attacks ONLY in training: 2
   ['spy', 'warezclient']

Attacks ONLY in test: 17
   ['xsnoop', 'worm', 'processtable', 'snmpguess', 'sendmail', 'saint', 'mscan', 'sqlattack', 'named', 'ps', 'snmpgetattack', 'xterm', 'udpstorm', 'httptunnel', 'xlock', 'apache2', 'mailbomb']

Common attacks: 21


In [103]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define categorical columns for encoding
categorical_columns = ['protocol_type', 'service', 'flag']

# Extract categorical values
df_categorical_values = df[categorical_columns]
testdf_categorical_values = test_df[categorical_columns]
df_categorical_values.head()

,protocol_type,service,flag
0,udp,other,SF
1,tcp,private,S0
2,tcp,http,SF
3,tcp,http,SF
4,tcp,private,REJ


In [104]:
# Generate column names for categorical encoding
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]

# Use combined unique services from both train and test sets
unique_service_train = set(df.service.unique())
unique_service_test = set(test_df.service.unique())
unique_service_combined = sorted(list(unique_service_train.union(unique_service_test)))
string2 = 'service_'
unique_service2 = [string2 + x for x in unique_service_combined]

# Flag columns
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]

# Combined columns for consistent encoding
dumcols = unique_protocol2 + unique_service2 + unique_flag2
testdumcols = dumcols

print(f"Total columns: {len(dumcols)}")
print(f"Protocol columns: {len(unique_protocol2)}")
print(f"Service columns: {len(unique_service2)}")
print(f"Flag columns: {len(unique_flag2)}")

print(f"\nServices only in train: {unique_service_train - unique_service_test}")
print(f"Services only in test: {unique_service_test - unique_service_train}")
print(f"Combined services: {len(unique_service_combined)}")

Total columns: 84
Protocol columns: 3
Service columns: 70
Flag columns: 11

Services only in train: {'red_i', 'harvest', 'http_2784', 'aol', 'urh_i', 'http_8001'}
Services only in test: set()
Combined services: 70


In [105]:
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

   protocol_type  service  flag
0              2       44     9
1              1       49     5
2              1       24     9
3              1       24     9
4              1       49     1


In [106]:
# OneHot encoding with proper handling of different categorical values
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Combine train and test categorical data to capture all possible categories
combined_categorical = pd.concat([df_categorical_values_enc, testdf_categorical_values_enc], ignore_index=True)

# Fit encoder on combined data to handle all categories consistently
enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
enc.fit(combined_categorical)

# Transform train and test sets
df_categorical_values_encenc = enc.transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc, columns=dumcols)

testdf_categorical_values_encenc = enc.transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc, columns=dumcols)

print(f"Train set categorical shape: {df_cat_data.shape}")
print(f"Test set categorical shape: {testdf_cat_data.shape}")
print(f"Column names match: {list(df_cat_data.columns) == list(testdf_cat_data.columns)}")

df_cat_data.head()

Train set categorical shape: (125972, 84)
Test set categorical shape: (22543, 84)
Column names match: True


,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [107]:
# Verify column structure consistency
print("Column structure verification:")
print(f"Train categorical shape: {df_cat_data.shape}")
print(f"Test categorical shape: {testdf_cat_data.shape}")
print(f"Columns match: {df_cat_data.shape[1] == testdf_cat_data.shape[1]}")

Column structure verification:
Train categorical shape: (125972, 84)
Test categorical shape: (22543, 84)
Columns match: True


In [108]:
# Final verification of column structure consistency
print("Column structure consistency achieved")

print(f"Final train categorical shape: {df_cat_data.shape}")
print(f"Final test categorical shape: {testdf_cat_data.shape}")
assert df_cat_data.shape[1] == testdf_cat_data.shape[1], "Column count mismatch!"
print("Verification complete - ready to proceed")

Column structure consistency achieved
Final train categorical shape: (125972, 84)
Final test categorical shape: (22543, 84)
Verification complete - ready to proceed


In [109]:
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=test_df.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=test_df.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)

(125972, 124)
(22543, 124)
(125972, 124)
(22543, 124)


In [110]:
# take label column
labeldf=newdf['attack']
labeldf_test=newdf_test['attack']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['attack'] = newlabeldf
newdf_test['attack'] = newlabeldf_test
print(newdf['attack'].head())

0    0
1    1
2    0
3    0
4    1
Name: attack, dtype: int64


/tmp/ipython-input-3327160078.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
/tmp/ipython-input-3327160078.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstor

In [111]:
to_drop_DoS = [2,3,4]
to_drop_Probe = [1,3,4]
to_drop_R2L = [1,2,4]
to_drop_U2R = [1,2,3]
DoS_df=newdf[~newdf['attack'].isin(to_drop_DoS)];
Probe_df=newdf[~newdf['attack'].isin(to_drop_Probe)];
R2L_df=newdf[~newdf['attack'].isin(to_drop_R2L)];
U2R_df=newdf[~newdf['attack'].isin(to_drop_U2R)];

#test
DoS_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_DoS)];
Probe_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_Probe)];
R2L_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_R2L)];
U2R_df_test=newdf_test[~newdf_test['attack'].isin(to_drop_U2R)];
print('Train:')
print('Dimensions of DoS:' ,DoS_df.shape)
print('Dimensions of Probe:' ,Probe_df.shape)
print('Dimensions of R2L:' ,R2L_df.shape)
print('Dimensions of U2R:' ,U2R_df.shape)
print('Test:')
print('Dimensions of DoS:' ,DoS_df_test.shape)
print('Dimensions of Probe:' ,Probe_df_test.shape)
print('Dimensions of R2L:' ,R2L_df_test.shape)
print('Dimensions of U2R:' ,U2R_df_test.shape)

Train:
Dimensions of DoS: (113269, 124)
Dimensions of Probe: (78998, 124)
Dimensions of R2L: (68337, 124)
Dimensions of U2R: (67394, 124)
Test:
Dimensions of DoS: (17170, 124)
Dimensions of Probe: (12132, 124)
Dimensions of R2L: (12596, 124)
Dimensions of U2R: (9778, 124)


In [112]:
# Split dataframes into X & Y
# assign X as a dataframe of feautures and Y as a series of outcome variables
X_R2L = R2L_df.drop('attack', axis=1)
Y_R2L = R2L_df.attack
X_Probe = Probe_df.drop('attack', axis=1)
Y_Probe = Probe_df.attack
X_DoS = DoS_df.drop('attack', axis=1)
Y_DoS = DoS_df.attack
X_U2R = U2R_df.drop('attack', axis=1)
Y_U2R = U2R_df.attack
# test set
X_R2L_test = R2L_df_test.drop('attack', axis=1)
Y_R2L_test = R2L_df_test.attack
X_Probe_test = Probe_df_test.drop('attack', axis=1)
Y_Probe_test = Probe_df_test.attack
X_DoS_test = DoS_df_test.drop('attack', axis=1)
Y_DoS_test = DoS_df_test.attack
X_U2R_test = U2R_df_test.drop('attack', axis=1)
Y_U2R_test = U2R_df_test.attack

In [113]:
colNames=list(X_R2L)
colNames_test=list(X_R2L_test)

In [114]:
from sklearn import preprocessing

# Feature scaling - fit on training data, transform both train and test
scaler_R2L = preprocessing.StandardScaler().fit(X_R2L)
X_R2L = scaler_R2L.transform(X_R2L)
X_R2L_test = scaler_R2L.transform(X_R2L_test)

In [115]:
# Verify standardization results
print("Standard deviation check (should be ~1.0):")
print(X_R2L.std(axis=0)[:10])  # Show first 10 features

Standard deviation check (should be ~1.0):
[1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]


In [116]:
from sklearn.feature_selection import SelectPercentile, f_classif
np.seterr(divide='ignore', invalid='ignore');
selector=SelectPercentile(f_classif, percentile=10)
X_newR2L = selector.fit_transform(X_R2L,Y_R2L)
X_newR2L.shape

/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [  4  16  44  45  47  48  49  50  51  52  55  58  59  63  64  65  67  68
  69  71  72  73  74  75  77  78  79  80  81  82  83  84  87  88  90  93
  94  97  99 100 101 108 109 110 111 115] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


(68337, 13)

In [117]:
true=selector.get_support()
newcolindex_R2L=[i for i, x in enumerate(true) if x]
newcolname_R2L=list( colNames[i] for i in newcolindex_R2L )
newcolname_R2L

['dst_bytes',
 'hot',
 'num_failed_logins',
 'is_guest_login',
 'dst_host_srv_count',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'level',
 'service_ftp',
 'service_ftp_data',
 'service_http',
 'service_imap4',
 'flag_RSTO']

In [118]:
from sklearn.feature_selection import RFE
clf = DecisionTreeClassifier(random_state=0)
rfe = RFE(estimator=clf, n_features_to_select=13, step=1)
rfe.fit(X_R2L, Y_R2L)
X_rfeR2L=rfe.transform(X_R2L)
true=rfe.support_
rfecolindex_R2L=[i for i, x in enumerate(true) if x]
rfecolname_R2L=list(colNames[i] for i in rfecolindex_R2L)

In [119]:
# Display selected features for R2L classification
print('Features selected for R2L classification:')
for feature in rfecolname_R2L:
    print(f"  - {feature}")

Features selected for R2L classification:
  - duration
  - src_bytes
  - dst_bytes
  - hot
  - logged_in
  - num_root
  - serror_rate
  - dst_host_count
  - dst_host_srv_count
  - dst_host_srv_diff_host_rate
  - dst_host_srv_serror_rate
  - level
  - service_ftp_data


In [120]:
print(X_rfeR2L.shape)

(68337, 13)


In [121]:
# Train decision tree classifier
clf_R2L = DecisionTreeClassifier(random_state=0)

# Fit classifier
clf_R2L.fit(X_R2L, Y_R2L)

DecisionTreeClassifier(random_state=0)

In [122]:
# Train classifier with RFE-selected features

clf_rfeR2L = DecisionTreeClassifier(random_state=0)
clf_rfeR2L.fit(X_rfeR2L, Y_R2L)

DecisionTreeClassifier(random_state=0)

In [123]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics

print("Evaluation Methodology Comparison:")
print("="*50)

# Cross-validation on training data (demonstrates inflated scores)
print("Cross-Validation on Training Data:")
accuracy = cross_val_score(clf_R2L, X_R2L, Y_R2L, cv=10, scoring='accuracy')
print("   CV Accuracy: %0.5f (+/- %0.5f)" % (accuracy.mean(), accuracy.std() * 2))

print("\nTrain-Test Evaluation:")
# Proper evaluation on test set
clf_R2L.fit(X_R2L, Y_R2L)
Y_pred = clf_R2L.predict(X_R2L_test)

print("   Test Accuracy:", metrics.accuracy_score(Y_R2L_test, Y_pred))
print("   Test Precision:", metrics.precision_score(Y_R2L_test, Y_pred, pos_label=3))
print("   Test Recall:", metrics.recall_score(Y_R2L_test, Y_pred, pos_label=3))
print("   Test F1:", metrics.f1_score(Y_R2L_test, Y_pred, pos_label=3))

Evaluation Methodology Comparison:
Cross-Validation on Training Data:
   CV Accuracy: 0.99921 (+/- 0.00067)

Train-Test Evaluation:
   Test Accuracy: 0.8028739282311845
   Test Precision: 0.9785714285714285
   Test Recall: 0.14246100519930677
   Test F1: 0.24871406959152798


In [124]:
X_R2L_test2 = X_R2L_test[:, rfecolindex_R2L]

Y_R2L_pred2 = clf_rfeR2L.predict(X_R2L_test2)

# Confusion matrix
print(pd.crosstab(Y_R2L_test, Y_R2L_pred2, rownames=['Actual attacks'], colnames=['Predicted attacks']))

# Final evaluation on test set (no CV here)
print("Test Accuracy:", metrics.accuracy_score(Y_R2L_test, Y_R2L_pred2))
print("Test Precision:", metrics.precision_score(Y_R2L_test, Y_R2L_pred2, pos_label=3))
print("Test Recall:", metrics.recall_score(Y_R2L_test, Y_R2L_pred2, pos_label=3))
print("Test F1:", metrics.f1_score(Y_R2L_test, Y_R2L_pred2, pos_label=3))

Predicted attacks     0    3
Actual attacks              
0                  9709    2
3                  2458  427
Test Accuracy: 0.8046999047316609
Test Precision: 0.9953379953379954
Test Recall: 0.14800693240901214
Test F1: 0.2576946288473144


In [125]:
# RFE Feature Selection Evaluation
print("RFE-based Feature Selection Performance:")
print("="*50)

# Train model with RFE-selected features
clf_rfeR2L.fit(X_R2L[:, rfecolindex_R2L], Y_R2L)
Y_pred_rfe = clf_rfeR2L.predict(X_R2L_test[:, rfecolindex_R2L])

print("Test Performance with RFE Features:")
print(f"Test Accuracy: {metrics.accuracy_score(Y_R2L_test, Y_pred_rfe):.5f}")
print(f"Test Precision: {metrics.precision_score(Y_R2L_test, Y_pred_rfe, pos_label=3):.5f}")
print(f"Test Recall: {metrics.recall_score(Y_R2L_test, Y_pred_rfe, pos_label=3):.5f}")
print(f"Test F1-Score: {metrics.f1_score(Y_R2L_test, Y_pred_rfe, pos_label=3):.5f}")

RFE-based Feature Selection Performance:
Test Performance with RFE Features:
Test Accuracy: 0.80470
Test Precision: 0.99534
Test Recall: 0.14801
Test F1-Score: 0.25769


In [126]:
X_R2L = np.array(X_R2L)
X_R2L_test = np.array(X_R2L_test)
#cnn-input
X_R2L = np.reshape(X_R2L, (X_R2L.shape[0], X_R2L.shape[1], 1))
X_R2L_test = np.reshape(X_R2L_test, (X_R2L_test.shape[0], X_R2L_test.shape[1], 1))
#lstm-input

In [127]:
Y_R2L = np.array(Y_R2L)
Y_R2L_test = np.array(Y_R2L_test)

In [128]:
# SMOTE Implementation for Class Imbalance
from imblearn.over_sampling import SMOTE
import numpy as np

print("Class distribution before SMOTE:")
print(f"  Normal (0): {np.sum(Y_R2L == 0)} samples")
print(f"  R2L Attack (3): {np.sum(Y_R2L == 3)} samples")
print(f"  Total samples: {len(Y_R2L)}")

# Apply SMOTE to balance the classes

# Reshape 3D CNN input back to 2D for SMOTE
X_R2L_2D = X_R2L.reshape(X_R2L.shape[0], -1)

# Apply SMOTE
smote = SMOTE(random_state=42, k_neighbors=5)
X_R2L_smote, Y_R2L_smote = smote.fit_resample(X_R2L_2D, Y_R2L)

print("Class distribution after SMOTE:")
print(f"  Normal (0): {np.sum(Y_R2L_smote == 0)} samples")
print(f"  R2L Attack (3): {np.sum(Y_R2L_smote == 3)} samples")
print(f"  Total samples: {len(Y_R2L_smote)}")

# Reshape back to 3D for CNN input
X_R2L_smote = X_R2L_smote.reshape(X_R2L_smote.shape[0], X_R2L.shape[1], X_R2L.shape[2])

print(f"\nFinal shapes after SMOTE:")
print(f"  X_R2L_smote shape: {X_R2L_smote.shape}")
print(f"  Y_R2L_smote shape: {Y_R2L_smote.shape}")

# Update the main variables to use SMOTE-enhanced data
X_R2L = X_R2L_smote
Y_R2L = Y_R2L_smote

Class distribution before SMOTE:
  Normal (0): 67342 samples
  R2L Attack (3): 995 samples
  Total samples: 68337
Class distribution after SMOTE:
  Normal (0): 67342 samples
  R2L Attack (3): 67342 samples
  Total samples: 134684

Final shapes after SMOTE:
  X_R2L_smote shape: (134684, 123, 1)
  Y_R2L_smote shape: (134684,)


In [129]:
# Convert training data (SMOTE-enhanced) from (0,3) to (0,1)
Y_R2L_binary = np.where(Y_R2L_smote == 3, 1, Y_R2L_smote)  # Convert 3 -> 1, keep 0 -> 0

# Convert test data from (0,3) to (0,1)
Y_R2L_test_binary = np.where(Y_R2L_test == 3, 1, Y_R2L_test)  # Convert 3 -> 1, keep 0 -> 0

# Update the variables for use in training and evaluation
Y_R2L = Y_R2L_binary  # Training labels (SMOTE-enhanced + binary)
Y_R2L_test = Y_R2L_test_binary  # Test labels (binary)


In [130]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam

In [131]:
# Focal Loss Implementation for Class Imbalance
import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2.0, alpha=0.25):
    """
    Focal Loss implementation for addressing class imbalance

    Args:
        gamma (float): Focusing parameter to down-weight easy examples
        alpha (float): Weighting parameter for rare class (positive class)

    Returns:
        loss function
    """
    def focal_loss_fixed(y_true, y_pred):
        # Ensure y_pred is in valid range [epsilon, 1-epsilon]
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate cross entropy
        ce_loss = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)

        # Calculate p_t
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)

        # Calculate alpha_t
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)

        # Calculate focal loss
        focal_loss = alpha_t * K.pow(1 - p_t, gamma) * ce_loss

        return K.mean(focal_loss)

    return focal_loss_fixed

# Calculate class weights for alpha parameter using SMOTE-enhanced BINARY data
from sklearn.utils.class_weight import compute_class_weight

print("Calculating focal loss parameters with SMOTE-enhanced BINARY data:")
print("IMPORTANT: Using binary labels (0/1) for proper focal loss calculation")

# Verify we have binary labels
unique_classes = np.unique(Y_R2L)
print(f"Unique classes in Y_R2L: {unique_classes}")

if len(unique_classes) == 2 and 0 in unique_classes and 1 in unique_classes:
    class_weights = compute_class_weight('balanced', classes=unique_classes, y=Y_R2L)
    print(f"Binary class distribution (after SMOTE and conversion):")
    print(f"  Normal (0): {np.sum(Y_R2L == 0)} samples")
    print(f"  R2L Attack (1): {np.sum(Y_R2L == 1)} samples")
    print(f"Class weights: {dict(zip(unique_classes, class_weights))}")

    # Set alpha based on class distribution (weight for positive/minority class)
    alpha_focal = class_weights[1] / (class_weights[0] + class_weights[1])
    print(f"Focal Loss alpha parameter: {alpha_focal:.3f}")
else:
    print("ERROR: Labels are not properly converted to binary (0/1)")
    print("Using default alpha = 0.5")
    alpha_focal = 0.5

# Create focal loss function with calculated alpha
focal_loss_fn = focal_loss(gamma=2.0, alpha=alpha_focal)

Calculating focal loss parameters with SMOTE-enhanced BINARY data:
IMPORTANT: Using binary labels (0/1) for proper focal loss calculation
Unique classes in Y_R2L: [0 1]
Binary class distribution (after SMOTE and conversion):
  Normal (0): 67342 samples
  R2L Attack (1): 67342 samples
Class weights: {np.int64(0): np.float64(1.0), np.int64(1): np.float64(1.0)}
Focal Loss alpha parameter: 0.500


In [132]:
# Simple Cost-Sensitive Learning Setup for R2L Detection
from sklearn.utils.class_weight import compute_class_weight

# Calculate enhanced class weights for better R2L detection
unique_classes = np.unique(Y_R2L)
base_weights = compute_class_weight('balanced', classes=unique_classes, y=Y_R2L)
cost_sensitive_weights = {0: base_weights[0], 1: base_weights[1] * 3.0}  # 3x penalty for missing R2L attacks

print(f"Cost-sensitive weights: {cost_sensitive_weights}")
print(f"Missing R2L attacks will be penalized 3x more than false alarms")

Cost-sensitive weights: {0: np.float64(1.0), 1: np.float64(3.0)}
Missing R2L attacks will be penalized 3x more than false alarms


In [133]:
import os

# Define a learning rate schedule
initial_learning_rate = 0.001
def lr_schedule(epoch):
    return initial_learning_rate * (0.1 ** (epoch // 10))

output_dir = 'Models'
os.makedirs(output_dir, exist_ok=True)
model_path = os.path.join(output_dir, 'R2L_base+focal_loss+SMOTE.h5')

print(f"Model will be saved to: {model_path}")

my_callbacks = [
    ModelCheckpoint(
        filepath=model_path,
        verbose=1,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1,
        min_delta=0.001
    ),
    LearningRateScheduler(lr_schedule, verbose=1)
]

Model will be saved to: Models/R2L_base+focal_loss+SMOTE.h5


In [134]:
# Lightweight CNN-LSTM Model Architecture for R2L Detection
model = Sequential()

# CNN layers for feature extraction
model.add(Conv1D(64, 3, padding="same", input_shape=(X_R2L.shape[1], 1), activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, 3, padding="same", activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# LSTM layer for sequence learning
model.add(LSTM(units=100, return_sequences=False, dropout=0.1))

# Output layer for binary classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile model with focal loss and cost-sensitive learning
opt = Adam(learning_rate=initial_learning_rate)
model.compile(
    loss=focal_loss_fn,  # Using focal loss for imbalanced data
    optimizer=opt,
    metrics=['accuracy', 'precision', 'recall']
)

print("Model Architecture:")
print(f"Input shape: {X_R2L.shape}")
print(f"Total parameters: {model.count_params():,}")
model.summary()

Model Architecture:
Input shape: (134684, 123, 1)
Total parameters: 116,661


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 123, 64)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 61, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 61, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 116,661 (455.71 KB)

 Trainable params: 116,661 (455.71 KB)

 Non-trainable params: 0 (0.00 B)

In [135]:
# Train/Validation Split for SMOTE-enhanced Data
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_R2L, Y_R2L,  # SMOTE-enhanced binary data
    test_size=0.2,
    stratify=Y_R2L,
    random_state=42
)


In [136]:
# Simple CNN-LSTM Training with Cost-Sensitive Learning
print("Training CNN-LSTM with cost-sensitive learning...")

history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_val, y_val),
    class_weight=cost_sensitive_weights,  # Cost-sensitive learning
    callbacks=my_callbacks,
    verbose=1
)

print("Training completed with cost-sensitive learning!")

Training CNN-LSTM with cost-sensitive learning...

Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/30
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9620 - loss: 0.0220 - precision: 0.9480 - recall: 0.9812
Epoch 1: val_accuracy improved from -inf to 0.99748, saving model to Models/R2L_base+focal_loss+SMOTE.h5


1684/1684 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9621 - loss: 0.0220 - precision: 0.9480 - recall: 0.9812 - val_accuracy: 0.9975 - val_loss: 0.0016 - val_precision: 0.9984 - val_recall: 0.9965 - learning_rate: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/30
1682/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9972 - loss: 0.0034 - precision: 0.9957 - recall: 0.9988
Epoch 2: val_accuracy improved from 0.99748 to 0.99885, saving model to Models/R2L_base+focal_loss+SMOTE.h5


1684/1684 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.9972 - loss: 0.0034 - precision: 0.9957 - recall: 0.9988 - val_accuracy: 0.9988 - val_loss: 7.7021e-04 - val_precision: 0.9979 - val_recall: 0.9999 - learning_rate: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/30
1681/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9984 - loss: 0.0020 - precision: 0.9973 - recall: 0.9995
Epoch 3: val_accuracy improved from 0.99885 to 0.99900, saving model to Models/R2L_base+focal_loss+SMOTE.h5


1684/1684 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9984 - loss: 0.0020 - precision: 0.9973 - recall: 0.9995 - val_accuracy: 0.9990 - val_loss: 7.4677e-04 - val_precision: 0.9980 - val_recall: 1.0000 - learning_rate: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/30
1678/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9983 - loss: 0.0019 - precision: 0.9974 - recall: 0.9992
Epoch 4: val_accuracy improved from 0.99900 to 0.99907, saving model to Models/R2L_base+focal_loss+SMOTE.h5


1684/1684 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9983 - loss: 0.0019 - precision: 0.9975 - recall: 0.9993 - val_accuracy: 0.9991 - val_loss: 5.7840e-04 - val_precision: 0.9981 - val_recall: 1.0000 - learning_rate: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/30
1680/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9990 - loss: 0.0014 - precision: 0.9983 - recall: 0.9997
Epoch 5: val_accuracy improved from 0.99907 to 0.99933, saving model to Models/R2L_base+focal_loss+SMOTE.h5


1684/1684 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - accuracy: 0.9990 - loss: 0.0014 - precision: 0.9983 - recall: 0.9997 - val_accuracy: 0.9993 - val_loss: 4.4836e-04 - val_precision: 0.9987 - val_recall: 1.0000 - learning_rate: 0.0010

Epoch 6: LearningRateScheduler setting learning rate to 0.001.
Epoch 6/30
1683/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9988 - loss: 0.0016 - precision: 0.9979 - recall: 0.9996
Epoch 6: val_accuracy did not improve from 0.99933
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.9988 - loss: 0.0016 - precision: 0.9979 - recall: 0.9996 - val_accuracy: 0.9991 - val_loss: 5.8127e-04 - val_precision: 0.9982 - val_recall: 1.0000 - learning_rate: 0.0010

Epoch 7: LearningRateScheduler setting learning rate to 0.001.
Epoch 7/30
1680/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9992 - loss: 0.0012 - precision: 0.9985 - recall: 0.9999
Epoch 7: val_accuracy improved from 0.99933 to 0.99959, saving model to Models/R2L_base+focal_loss+SMOTE.h5


1684/1684 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9992 - loss: 0.0012 - precision: 0.9985 - recall: 0.9999 - val_accuracy: 0.9996 - val_loss: 3.7916e-04 - val_precision: 0.9992 - val_recall: 1.0000 - learning_rate: 0.0010

Epoch 8: LearningRateScheduler setting learning rate to 0.001.
Epoch 8/30
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9993 - loss: 0.0010 - precision: 0.9988 - recall: 0.9998
Epoch 8: val_accuracy did not improve from 0.99959
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - accuracy: 0.9993 - loss: 0.0010 - precision: 0.9988 - recall: 0.9998 - val_accuracy: 0.9990 - val_loss: 5.9735e-04 - val_precision: 0.9981 - val_recall: 1.0000 - learning_rate: 0.0010

Epoch 9: LearningRateScheduler setting learning rate to 0.001.
Epoch 9/30
1682/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9993 - loss: 8.6985e-04 - precision: 0.9988 - recall: 0.9998
Epoch 9: val_accuracy did not improve from 0.99959
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accurac

1684/1684 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.9995 - loss: 7.9902e-04 - precision: 0.9990 - recall: 1.0000 - val_accuracy: 0.9997 - val_loss: 3.4625e-04 - val_precision: 0.9993 - val_recall: 1.0000 - learning_rate: 1.0000e-04

Epoch 12: LearningRateScheduler setting learning rate to 0.0001.
Epoch 12/30
1679/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9996 - loss: 6.2221e-04 - precision: 0.9993 - recall: 1.0000
Epoch 12: val_accuracy did not improve from 0.99967
1684/1684 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.9996 - loss: 6.2211e-04 - precision: 0.9993 - recall: 1.0000 - val_accuracy: 0.9996 - val_loss: 3.2744e-04 - val_precision: 0.9991 - val_recall: 1.0000 - learning_rate: 1.0000e-04

Epoch 13: LearningRateScheduler setting learning rate to 0.0001.
Epoch 13/30
1680/1684 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9998 - loss: 3.9424e-04 - precision: 0.9996 - recall: 1.0000
Epoch 13: val_accuracy did not improve from 0.99967
1684/1684 ━━━━━━━━━━━━━━━━

In [143]:
# Simple Prediction and Evaluation
y_pred_proba = model.predict(X_R2L_test, verbose=0)
y_pred = (y_pred_proba > 0.004).astype(int).flatten()

# Results
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(Y_R2L_test, y_pred)
precision = precision_score(Y_R2L_test, y_pred)
recall = recall_score(Y_R2L_test, y_pred)
f1 = f1_score(Y_R2L_test, y_pred)

print("R2L Attack Detection Results (Cost-Sensitive Learning):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print(f"\nConfusion Matrix:")
print(confusion_matrix(Y_R2L_test, y_pred))

print(f"\nClassification Report:")
print(classification_report(Y_R2L_test, y_pred, target_names=['Normal', 'R2L Attack'], digits=4))

R2L Attack Detection Results (Cost-Sensitive Learning):
Accuracy: 0.9453
Precision: 0.8544
Recall: 0.9175
F1-Score: 0.8848

Confusion Matrix:
[[9260  451]
 [ 238 2647]]

Classification Report:
              precision    recall  f1-score   support

      Normal     0.9749    0.9536    0.9641      9711
  R2L Attack     0.8544    0.9175    0.8848      2885

    accuracy                         0.9453     12596
   macro avg     0.9147    0.9355    0.9245     12596
weighted avg     0.9473    0.9453    0.9460     12596

